In [1]:
import pandas as pd
import jieba
import os
import pyLDAvis
import pyLDAvis.lda_model
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation

# 转化为python文件：jupyter nbconvert --to script csv_lda.ipynb （需pip install nbconvert）   
# pip install scikit-learn==0.20.0
# conda install -c conda-forge pyldavis
# https://stackoverflow.com/questions/76836793/jupyter-notebook-cannot-import-pyldavis-sklearn



In [2]:
current_path = os.getcwd()  
base_path = os.path.dirname(current_path) 
cn_stopwords=os.path.join(base_path, "data/cn_stopwords.txt")
csv_path = os.path.join(base_path, "data/专利demo.xlsx")
df = pd.read_excel(csv_path, engine='openpyxl')
df.head()

,行业大类编码,行业大类名称,省份编码,省份名称,企业id,统一社会信用diamante,公开号,公开日期,申请号,专利类型,专利类型代码,申请日期,专利名称,主分类号,分类号,申请人,发明人,代理机构,代理人,摘要
0,big_industry_code,big_industry_name,province_code,province_name,company_id,company_credit_code,publication_no,publication_date,apply_no,patent_type,patent_type_code,apply_date,patent_name,main_classification_no,classification_no,applicant,inventor_ame,agency_org,agency_name,abstract
1,06,煤炭开采和洗选业,110000,北京市,CAICT_COM_100006_C2A9FF4B9A889A4B168E99F8EB845B2E,9111000010000085X6,CN116687362A,2023-09-05,CN202310711693.4,发明专利,11,2023-06-15,一种岗前健康快速筛查系统及方法,A61B5/0205,A61B5/0205;A61B5/16;A61B5/00,中国中煤能源集团有限公司,周亚东;周应江;闫建华;王云泉;薛泽宇;赵学江;杨轲;赵玉宝;郭超荣;陈洪林;杜大文;葛庆岗...,北京集佳知识产权代理有限公司,张雪娇,本申请提供一种岗前健康快速筛查系统及方法，方法包括：显示健康快速筛查界面；其中，健康快速筛查...
2,06,煤炭开采和洗选业,110000,北京市,CAICT_COM_100006_2D1489FA41B9846088A9C422C03CC68D,91110000592360113C,CN116699688A,2023-09-05,CN202211731131.8,发明专利,11,2022-12-30,一种走滑断层识别方法及系统,G01V1/30,G01V1/30;G01V1/28,华能煤业有限公司,袁浩伟;荆铁亚;刘练波;周娟;赵文韬;张健;朱明宇;尹玉龙;李义朝;高文,西安通大专利代理有限责任公司,贺小停,本申请提供的一种走滑断层识别方法及系统，利用三维地震资料、构造图等资料，获得待测断层倾角以及...
3,06,煤炭开采和洗选业,110000,北京市,CAICT_COM_100006_C2A9FF4B9A889A4B168E99F8EB845B2E,9111000010000085X6,CN116596320A,2023-08-15,CN202310835476.6,发明专利,11,2023-07-10,一种煤矿作业人员的风险评估方法及系统,G06Q10/0635,G06Q10/0635;G16H50/30;G06Q50/02,中国中煤能源集团有限公司,周亚东;周应江;闫建华;王云泉;薛泽宇;赵学江;杨轲;赵玉宝;郭超荣;陈洪林;杜大文;葛庆岗...,北京集佳知识产权代理有限公司,张雪娇,本发明提供一种煤矿作业人员的风险评估方法及系统，通过获取待评估煤矿作业人员的生理参数和心理参...
4,06,煤炭开采和洗选业,110000,北京市,CAICT_COM_100006_2D1489FA41B9846088A9C422C03CC68D,91110000592360113C,CN116517586A,2023-08-01,CN202310441710.7,发明专利,11,2023-04-23,一种深井巷道防冲击地压支护装置,E21D11/10,E21D11/10;E21D11/00;E21D21/00;E21D20/02,华能煤业有限公司,汪占领;杜金洲;王志超;杨云斌;程利兴;赵亮;王港盛;高健勋;石蒙;李汉汉;郭罡业;肖运智;...,郑州科硕专利代理事务所(普通合伙),范增哲,本发明公开了一种深井巷道防冲击地压支护装置，本发明涉及深井巷道支护技术领域，包括防变形机构，...


In [3]:
df.shape

(26535, 20)

In [4]:
def chinese_word_cut(mytext):
    return " ".join(jieba.cut(mytext))

批量处理数据框里面的全部文本（正文）信息了

In [5]:
df["summary_cutted"] = df["摘要"].apply(chinese_word_cut)

Building prefix dict from the default dictionary ...
Loading model from cache /var/folders/4m/3yfnhmtx0vx1d01xd38_4r5r0000gn/T/jieba.cache
Loading model cost 0.286 seconds.
Prefix dict has been built successfully.


In [6]:
df.summary_cutted.head()

0                                             abstract
1    本 申请 提供 一种 岗前 健康 快速 筛查 系统 及 方法 ， 方法 包括 ： 显示 健康...
2    本 申请 提供 的 一种 走 滑 断层 识别方法 及 系统 ， 利用 三维 地震 资料 、 ...
3    本发明 提供 一种 煤矿 作业 人员 的 风险 评估 方法 及 系统 ， 通过 获取 待 评...
4    本发明 公开 了 一种 深井 巷道 防 冲击 地压 支护 装置 ， 本发明 涉及 深井 巷道...
Name: summary_cutted, dtype: object

加载中文停用词表

In [7]:
# 读取停用词文本文件  
with open(cn_stopwords, 'r', encoding='utf-8') as f:  
    stop_words = [line.strip() for line in f.readlines()]


我们不希望处理所有词汇。因为一来处理时间太长，二来那些很不常用的词汇对我们的主题抽取意义不大。所以这里做了个限定，只从文本中提取1000个最重要的特征关键词，然后停止。开始关键词提取和向量转换过程：先设定为5个分类。

In [8]:
n_features = 1000
tf_vectorizer = CountVectorizer(strip_accents = 'unicode',
                                max_features=n_features,
                                stop_words=stop_words,
                                max_df = 0.5,
                                min_df = 10)
tf = tf_vectorizer.fit_transform(df.summary_cutted)

In [9]:
n_topics = 10
lda = LatentDirichletAllocation(n_components=n_topics, max_iter=50,
                                learning_method='online',
                                learning_offset=50.,
                                random_state=0)

In [10]:
lda.fit(tf)

LatentDirichletAllocation(learning_method='online', learning_offset=50.0,
                          max_iter=50, random_state=0)

In [11]:
def print_top_words(model, feature_names, n_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print("Topic #%d:" % topic_idx)
        print(" ".join([feature_names[i]
                        for i in topic.argsort()[:-n_top_words - 1:-1]]))
    print()

In [12]:
n_top_words = 20  
tf_feature_names = tf_vectorizer.get_feature_names_out()
# tf_feature_names = tf_vectorizer.get_feature_names_out() 新版本sklern
print_top_words(lda, tf_feature_names, n_top_words)

Topic #0:
所述 制备 催化剂 材料 具有 金属 应用 组分 表面 公开 提供 涉及 sup 活性 10 载体 性能 重量 分子筛 得到
Topic #1:
显示 所述 信号 基板 电路 发光 区域 位于 至少 储能 显示装置 光伏 提供 充电 控制 设置 方向 导电 晶体管 电池
Topic #2:
测量 进行 步骤 地震 压力 确定 数据 模拟 计算 储层 曲线 频率 地层 试验 提供 参数 速度 不同 测井 注入
Topic #3:
模型 预测 计算 进行 优化 负荷 参数 得到 确定 评价 构建 数据 建立 步骤 评估 功率 目标 系统 配电网 指标
Topic #4:
sub 电压 电极 衬底 结构 电流 器件 形成 线路 栅极 输电 基底 隔离 半导体 用电 介质 配电 放电 电阻 烟气
Topic #5:
数据 所述 信息 进行 目标 像素 设备 获取 图像 确定 对应 装置 终端 用户 电力 提供 得到 申请 处理 公开
Topic #6:
所述 连接 设置 装置 组件 安装 驱动 固定 机构 结构 设有 面板 用于 支撑 之间 一侧 能够 公开 移动 转动
Topic #7:
技术 进行 涉及 领域 问题 解决 实现 提高 采用 现有 过程 生产 加工 工艺 属于 有效 降低 能够 作业 提供
Topic #8:
进行 反应 得到 处理 所述 混合 装置 分离 气体 反应器 加氢 步骤 加热 系统 循环 公开 催化剂 原料 进入 出口
Topic #9:
模块 系统 单元 控制 检测 装置 运行 故障 用于 电网 状态 监测 进行 设备 测试 输出 实现 连接 采集 传感器



In [13]:
data = pyLDAvis.lda_model.prepare(lda, tf, tf_vectorizer)
pyLDAvis.show(data, local=False)#local=False避免报错，会在显示器打开

Serving to http://127.0.0.1:8888/    [Ctrl-C to exit]


127.0.0.1 - - [02/Nov/2023 03:31:48] "GET / HTTP/1.1" 200 -
